In [2]:
import statsapi
import requests
import json
BASE_URL = "https://statsapi.mlb.com/api"

from datetime import datetime
today = datetime.today().strftime('%Y-%m-%d')

sched = statsapi.schedule(date=today)
print(sched)

games = []


for game in sched:
    games.append({"away" : {'id': game['away_id'], 'sp' : game['away_probable_pitcher']}, "home": {'id': game['home_id'], 'sp' : game['away_probable_pitcher']}})
# # for game in sched:
games
    
def check_div_index(div_id, league_id):
    div_index = -1
    if league_id == 103:
        # al team
        if div_id == 201:
            # al east
            div_index = 0
        elif div_id == 202:
            # al central
            div_index = 1
        elif div_id == 200:
            # al west
            div_index = 2
    elif league_id == 104:
        # nl team
        if div_id == 204:
            # nl east
            div_index = 0
        elif div_id == 205:
            # nl central
            div_index = 1
        elif div_id == 203:
            # nl west
            div_index = 2
    return div_index




[{'game_id': 745896, 'game_datetime': '2024-06-12T17:10:00Z', 'game_date': '2024-06-12', 'game_type': 'R', 'status': 'Final', 'away_name': 'Colorado Rockies', 'home_name': 'Minnesota Twins', 'away_id': 115, 'home_id': 142, 'doubleheader': 'N', 'game_num': 1, 'home_probable_pitcher': 'Pablo López', 'away_probable_pitcher': 'Austin Gomber', 'home_pitcher_note': '', 'away_pitcher_note': '', 'away_score': 9, 'home_score': 17, 'current_inning': 9, 'inning_state': 'Top', 'venue_id': 3312, 'venue_name': 'Target Field', 'national_broadcasts': ['MLBN (out-of-market only)'], 'series_status': 'MIN wins 2-1', 'winning_team': 'Minnesota Twins', 'losing_team': 'Colorado Rockies', 'winning_pitcher': 'Pablo López', 'losing_pitcher': 'Austin Gomber', 'save_pitcher': None, 'summary': '2024-06-12 - Colorado Rockies (9) @ Minnesota Twins (17) (Final)'}, {'game_id': 745978, 'game_datetime': '2024-06-12T18:10:00Z', 'game_date': '2024-06-12', 'game_type': 'R', 'status': 'Final', 'away_name': 'Toronto Blue Ja

In [57]:
# print(games)

# print(games)
for game in games:
    # print(game)
    try:
        response_away = requests.get(f"{BASE_URL}/v1/teams/{game['away']['id']}")
        game['away']['div_id'] = response_away.json()['teams'][0]['division']['id']
        game['away']['league_id'] = response_away.json()['teams'][0]['league']['id']
        response_home = requests.get(f"{BASE_URL}/v1/teams/{game['home']['id']}")
        game['home']['div_id'] = response_home.json()['teams'][0]['division']['id']
        game['home']['league_id'] = response_home.json()['teams'][0]['league']['id']

    except requests.exceptions.RequestException as e:
        print(e)

# games

for game in games:
    away_div_id = game['away']['div_id']
    home_div_id = game['home']['div_id']
    away_league_id = game['away']['league_id']
    home_league_id = game['home']['league_id']
    game['away']['div_index'] = check_div_index(away_div_id, away_league_id)
    game['home']['div_index'] = check_div_index(home_div_id, home_league_id)

games


[{'away': {'id': 115, 'div_id': 203, 'league_id': 104, 'div_index': 2},
  'home': {'id': 142, 'div_id': 202, 'league_id': 103, 'div_index': 1}},
 {'away': {'id': 141, 'div_id': 201, 'league_id': 103, 'div_index': 0},
  'home': {'id': 158, 'div_id': 205, 'league_id': 104, 'div_index': 1}},
 {'away': {'id': 117, 'div_id': 200, 'league_id': 103, 'div_index': 2},
  'home': {'id': 137, 'div_id': 203, 'league_id': 104, 'div_index': 2}},
 {'away': {'id': 133, 'div_id': 200, 'league_id': 103, 'div_index': 2},
  'home': {'id': 135, 'div_id': 203, 'league_id': 104, 'div_index': 2}},
 {'away': {'id': 144, 'div_id': 204, 'league_id': 104, 'div_index': 0},
  'home': {'id': 110, 'div_id': 201, 'league_id': 103, 'div_index': 0}},
 {'away': {'id': 120, 'div_id': 204, 'league_id': 104, 'div_index': 0},
  'home': {'id': 116, 'div_id': 202, 'league_id': 103, 'div_index': 1}},
 {'away': {'id': 112, 'div_id': 205, 'league_id': 104, 'div_index': 1},
  'home': {'id': 139, 'div_id': 201, 'league_id': 103, 'di

In [59]:

for game in games:
    away_div_index = game['away']['div_index']
    away_league_id = game['away']['league_id']
    away_team_id = game['away']['id']
    home_div_index = game['home']['div_index']
    home_league_id = game['home']['league_id']
    home_team_id = game['home']['id']
    home_response = requests.get(f"https://statsapi.mlb.com/api/v1/standings?leagueId={home_league_id}&season=2024&standingsTypes=regularSeason")
    away_response = requests.get(f"https://statsapi.mlb.com/api/v1/standings?leagueId={away_league_id}&season=2024&standingsTypes=regularSeason")
    if away_response.status_code == 200:
        away_json = away_response.json()
        away_division_data = away_json['records'][away_div_index]['teamRecords']
        for team in away_division_data:
            if team['team']['id'] == away_team_id:
                game['away']['team_data'] = team
    if home_response.status_code == 200:
        home_json = home_response.json()
        home_division_data = home_json['records'][home_div_index]['teamRecords']
        for team in home_division_data:
            if team['team']['id'] == home_team_id:
                game['home']['team_data'] = team




